# 1.데이터 다운받기

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2.데이터 보여주기

In [ ]:
train = pd.read_csv("../input/titanic/train.csv")
train.head()

In [ ]:
test = pd.read_csv("../input/titanic/test.csv")
test.head()

# 3.빈 데이터 있는지 확인

In [ ]:
train.info()

In [ ]:
train.info()

# age,cabin,embarked 이 세개의 값에 비어있는 값이 있었다.
survived : 생존=1, 죽음=0
pclass : 승객 등급. 1등급=1, 2등급=2, 3등급=3
sibsp : 함께 탑승한 형제 또는 배우자 수
parch : 함께 탑승한 부모 또는 자녀 수
ticket : 티켓 번호
cabin : 선실 번호
embarked : 탑승장소 S=Southhampton, C=Cherbourg, Q=Queenstown

.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [ ]:
def chart(feature):
    survived = train[train['Survived']==1][feature].value_counts()
    dead = train[train['Survived']==0][feature].value_counts()
    df = pd.DataFrame([survived,dead])
    df.index = ['Survived','Dead']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

특성에 따라 생존확률을 보기 위해 데이터를 시각화하려고 했다
그러기 위해서 두개의 막대차트로 표시하는 함수를 정의해서 특성을 하나씩 넣어보았다.

다른 특성과 달리 age의 같은경우는 그냥 시각화 하기에는 너무 다양하므로 대체적인 나잇대를 분류해보았다.

In [ ]:
train["Age"] = train["Age"].fillna(-0.5)
test["Age"] = test["Age"].fillna(-0.5)
bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
train['AgeGroup'] = pd.cut(train["Age"], bins, labels = labels)
test['AgeGroup'] = pd.cut(test["Age"], bins, labels = labels)

chart('AgeGroup')

In [ ]:
chart('Sex')

여성이 확률적으로 더욱 많이 살아남았다

In [ ]:
chart('Pclass')

1등급클래스를 탄 승객이 살아남은 비율이 더 높았다 등급은 생존에 영향을 준다고 생각했다!!
1등급 클래스는 살 가능성이 더 높고 
3등급 클래스는  죽을 가능성이 더 높다!

In [ ]:
chart('SibSp')

확률적으로 혼자인 사람은 형제가 있는 사람보다 죽을 확률이 높다
하지만 형제가 2명이상있는 대체적으로 사람들도 죽을 확률이 높다

In [ ]:
chart('Parch')

위에 sibsp와 유사한 결과이다 가족이 아예없는것도 너무 많은것도 죽을 확률이 높은것 같다.

In [ ]:
chart('Embarked')

c선착장에 탄 사람들만 산 사람의 비율이 더 높았다-> (예상)c선착장에 부자가 많아 1st class 탑승자가 많았던걸까?

# 4. Feature engineering
Feature Engineering은 데이터에 대한 도메인 지식을 사용하여 기계 학습 알고리즘을 작동시키는 Feature을 만드는 과정이다
이 과정이 매우 중요하다는 것을 알았으므로 많은 예시와 가정을 생각했다

4.1. age값의 경우 빈 값이 있으므로 채워주었어야 했다 그 빈 값을 평균으로 숫자를 내서 넣는 방법도 있었지만
age가 빈 값은 다른 정보 즉 이름과 성별은 데이터가 있었기 떄문에 이 두가지 데이터로 나이를 예측해서 넣는 방법이 정확도가 높을것 같아서 한번 시도해 보았다.

In [ ]:
combine = [train, test] #트레인과 테스트데이터를 동시에 변경시키기위해 코딩했다

for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train['Title'], train['Sex'])

너무 다양한 성들이 있으므로 분류를 하기위해서 위에서 본 데이터를 참고하여 
위에서 데이터를 시각화 시켰을떄 1등급좌석을 탑승한 사람들은 살아남을 확률이 높았으므로 고위층 즉 돈이 많은 사람들은 1등급을 탔을것이라고 생각하고 다른 성과 분류했다
부모 즉 아이가 있는 부인이라는 성도 가족이 있을경우 살아남을 확률이 더 높았으므로 따로 분류하였다
나머지는 여자 남자,드문,아기 로 분류하였다

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Capt', 'Col',
    'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    
    dataset['Title'] = dataset['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Royal": 5, "Rare": 6}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train.head()

나잇대에 맞는 숫자값을 채워 넣어 빈 값이 없게 하였다!

In [ ]:
mr_age = train[train["Title"] == 1]["AgeGroup"].mode() #Young Adult
miss_age = train[train["Title"] == 2]["AgeGroup"].mode() #Student
mrs_age = train[train["Title"] == 3]["AgeGroup"].mode() #Adult
master_age = train[train["Title"] == 4]["AgeGroup"].mode() #Baby
royal_age = train[train["Title"] == 5]["AgeGroup"].mode() #Adult
rare_age = train[train["Title"] == 6]["AgeGroup"].mode() #Adult

age_title_mapping = {1: "Young Adult", 2: "Student", 3: "Adult", 4: "Baby", 5: "Adult", 6: "Adult"}

for x in range(len(train["AgeGroup"])):
    if train["AgeGroup"][x] == "Unknown":
        train["AgeGroup"][x] = age_title_mapping[train["Title"][x]]
        
for x in range(len(test["AgeGroup"])):
    if test["AgeGroup"][x] == "Unknown":
        test["AgeGroup"][x] = age_title_mapping[test["Title"][x]]

In [ ]:

age_mapping = {'Baby': 1, 'Child': 2, 'Teenager': 3, 'Student': 4, 'Young Adult': 5, 'Adult': 6, 'Senior': 7}
train['AgeGroup'] = train['AgeGroup'].map(age_mapping)
test['AgeGroup'] = test['AgeGroup'].map(age_mapping)

train.head()


name,age는 이제 필요없어졌으므로 삭제한다.

In [ ]:
train = train.drop(['Name'], axis = 1)
test = test.drop(['Name'], axis = 1)

train = train.drop(['Age'], axis = 1)
test = test.drop(['Age'], axis = 1)

4.2. 성별은 이미 두가지로 완벽하게 분류되어 있으므로 숫자로만 변경한다.

In [ ]:
sex_mapping = {"male": 0, "female": 1}
train['Sex'] = train['Sex'].map(sex_mapping)
test['Sex'] = test['Sex'].map(sex_mapping)

train.head()

In [ ]:
chart('Sex')

agegroup에 값이 잘 들어갔는지 확인했다

In [ ]:
test.info()

In [ ]:
train.info()

4.3. embrked 값을 채우기 위해 도시별로 부유한 사람의 비율이 크게 차이나는지 살펴보려고 데이터를 시각화하였다.

In [ ]:
Pclass1 = train[train['Pclass']==1]['Embarked'].value_counts()
Pclass2 = train[train['Pclass']==2]['Embarked'].value_counts()
Pclass3 = train[train['Pclass']==3]['Embarked'].value_counts()
df = pd.DataFrame([Pclass1, Pclass2, Pclass3])
df.index = ['1st class', '2nd class', '3rd class']
df.plot(kind='bar', stacked=True, figsize=(10,5))

Q 도시에서 탄 사람들은 1등급이 거의 없고 S 도시에서 탑승한 사람들은 모두 존재하므로 빈 embarked  값이 2개뿐이므로 평균적인 s를 넣어도 상관없을것 같아서  s를 넣기로 했다.
 그리고 숫자로 변환시켰다.

In [ ]:
train_test_data = [train, test]
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
embarked_mapping = {'S':0, 'C':1, 'Q':2}
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)

4.4.fare의 경우 클래스와 연관이 크므로 클래스의 값은 빈 값이 없으므로 class를 참고하면 된다고 생각했다

In [ ]:
train["Fare"].fillna(train.groupby('Pclass')['Fare'].transform('median'), inplace=True)
test["Fare"].fillna(test.groupby('Pclass')['Fare'].transform('median'), inplace=True)

In [ ]:
facet = sns.FacetGrid(train, hue="Survived", aspect=4)
facet.map(sns.kdeplot, 'Fare', shade=True)
facet.set(xlim=(0, train['Fare'].max()))
facet.add_legend()

plt.show()

실제로 싼티켓은 비싼 티켓보다 사망을 많이 했다는것을 알 수 있다.

In [ ]:
for dataset in train_test_data:
    dataset.loc[ dataset['Fare'] <=17, 'Fare'] = 0,
    dataset.loc[(dataset['Fare'] > 17) & (dataset['Fare'] <=30), 'Fare'] = 1,
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <=100), 'Fare'] = 2,
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3,

In [ ]:
train.head()

4.5.cabin의 값을 정리하려고 했는데 너무 값이 다양해서 앞에 알파벳으로만 분류해보았다

In [ ]:
train.Cabin.value_counts()

In [ ]:
for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].str[:1]

In [ ]:
Pclass1 = train[train['Pclass']==1]['Cabin'].value_counts()
Pclass2 = train[train['Pclass']==2]['Cabin'].value_counts()
Pclass3 = train[train['Pclass']==3]['Cabin'].value_counts()
df = pd.DataFrame([Pclass1, Pclass2, Pclass3])
df.index = ['1st class', '2nd class', '3rd class']
df.plot(kind='bar', stacked=True, figsize=(10,5))

1등급에는 ABCDET, 2등급에는 DEF, 3등급은 EFG로 구성되어 있었다.
분류하기 편해졌으므로 이걸 숫자로 바꾸려고 했다.

범위를 비슷하게 주기 위해서 feature scaling를 사용하여 분류하였다.
Cabin의 missing field는 1등급 2등급 3등급 클래스와 관련이 있는게 위에서 보여졌기 떄문에 각 클래스별 cabin의 중간값을 missing value에 넣어주었다.

In [ ]:
cabin_mapping = {'A':0, 'B':0.4, 'C':0.8, 'D':1.2, 'E':1.6, 'F':2, 'G':2.4, 'T': 2.8}
for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)

In [ ]:
train['Cabin'].fillna(train.groupby('Pclass')['Cabin'].transform('median'), inplace=True)
test['Cabin'].fillna(test.groupby('Pclass')['Cabin'].transform('median'), inplace=True)

In [ ]:
train.info()

4.6.familysize값과 sibsb값은 사망률에 대한 관계가 비슷해서 두가지를 그냥 합쳐버리면 되지 않을까라고 생각했다. 중요한 점은 혼자인지 함꼐인지 같았으므로 그걸 중심으로 잡았다.

In [ ]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

In [ ]:
facet = sns.FacetGrid(train, hue="Survived", aspect=4)
facet.map(sns.kdeplot, 'FamilySize', shade=True)
facet.set(xlim=(0, train['FamilySize'].max()))
facet.add_legend()

그래프를 통해 혼자 탔을경우 가족이나 자식이 있을경우보다 사망률이 많이 높다는것을 실제로 볼 수 있었다

familysize도 숫자로 변형시켜주었다.

In [ ]:
family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
for dataset in train_test_data:
    dataset['FamilySize'] = dataset['FamilySize'].map(family_mapping)

4.7Ticket, SibSp와 Parch는 사망률에 관계가 없어보여서 뺴주었다.

In [ ]:
features_drop = ['Ticket', 'SibSp', 'Parch']
train = train.drop(features_drop, axis=1)
test = test.drop(features_drop, axis=1)
train = train.drop(['PassengerId'], axis=1)

In [ ]:
train_data = train.drop('Survived', axis=1)
target = train['Survived']

train_data.shape, target.shape

In [ ]:
train_data.head(10)

# 5.모델링

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import numpy as np

In [ ]:
train.info()

# 6.Classifier로 분류하기
어떤것이 제일 정확도가 높을지 알기위해 여러가지 방법을 사용해보았다.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

# 6.1. kNN Classifier

In [ ]:
#  k=13 개의 네이버로 설정했다,
clf = KNeighborsClassifier(n_neighbors = 13)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# kNN Score
round(np.mean(score)*100, 2)

# 6.2 Decision Tree

In [ ]:
clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# decision tree Score
round(np.mean(score)*100, 2)

# 6.3 Ramdom Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=13)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# Random Forest Score
round(np.mean(score)*100, 2)

# 6.4 Naive Bayes

In [ ]:
clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# Naive Bayes Score
round(np.mean(score)*100, 2)

# 6.5 SVM

In [ ]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100,2)

여러가지 방법을 사용했을떄 svm의 값이 가장크게 나왔으므로 svm을 사용했다. 
svm으로 테스트를 해보자!

# 7.Test

In [ ]:
clf = SVC()
clf.fit(train_data, target)

test_data = test.drop("PassengerId", axis=1).copy()
prediction = clf.predict(test_data)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": prediction
    })

submission.to_csv('submission.csv', index=False)

In [ ]:
submission = pd.read_csv('submission.csv')
submission.head()

# 8.결과분석
생각보다 낮아서 너무 실망스러웠다 대체 문제점이 뭔지 감도 안잡힌다
책을 참고하여 처음 시도했을 떄는 디씨전트리와 랜덤포레스트로 예측하는 방법을 사용해 보았는데 결과는 80%를 넘지 못해서
다른 방식을 사용했지만  트레이닝 데이터를 쪼개서 학습테스트한 결과의 정확도는 엄청나게 높았지만  실제 캐글의 테스트 데이터로 예측한 결과의 정확도는 매우 낮은 편으로 나와서
이유를 도저히 모르겠어서 다른 사람의 notebook을 참고한 결과 데이터 분석 작업의 중요성을 알아서
이번 notebook에서는 데이터분석부터 시작해서 여러가지 방법을 참고해보았는데 80프로도 넘지 못했다. 내가 데이터를 분류한것이 잘못되었던것 같다 내 실력으로는 어떤 데이터 분류를 잘못했는지 전혀 모르겠다. 내가 어떤 부분에서 놓쳤는지 모르겠다 그래도 처음보다는 확률이 높아져서 만족스럽기도 했다 


# 9.참고한 자료
1. kaggle-notebook Titanic Survival Predictions (Beginner) -https://www.kaggle.com/nadintamer/titanic-survival-predictions-beginner
2.Mukesh ChapagainTitanic Solution: A Beginner's Guide -https://www.kaggle.com/chapagain/titanic-solution-a-beginner-s-guide?scriptVersionId=1473689
3.How to score 0.8134 in Titanic Kaggle Challenge - https://www.ahmedbesbes.com/blog/kaggle-titanic-competition
4.Titanic: factors to survive - https://olegleyz.github.io/titanic_factors.html
5.Titanic Survivors Dataset and Data Wrangling - https://www.codeastar.com/data-wrangling/